In [1]:
import yfinance as yf
import pandas as pd

entry_time = "09:30"
exit_time = "15:00"

start_date = "2025-11-14"
end_date = "2025-11-20"
stock_name = "SBIN.NS"

# Download 1-minute data
df = yf.download(stock_name, interval="1m", period="7d")
df.index = df.index.tz_localize(None)

df = df.loc[start_date:end_date]

# Reset MultiIndex columns (keep only single level)
df.columns = df.columns.get_level_values(0)

df["Date"] = df.index.date
df["Time"] = df.index.time

results = []

entry_dt = pd.to_datetime(entry_time).time()
exit_dt = pd.to_datetime(exit_time).time()

for d in sorted(df["Date"].unique()):
    day_data = df[df["Date"] == d]

    # Find nearest earlier candles
    entry_rows = day_data[day_data["Time"] <= entry_dt]
    exit_rows = day_data[day_data["Time"] <= exit_dt]

    if entry_rows.empty or exit_rows.empty:
        print(f"No valid candles available for {d}, skipping...")
        continue

    entry_row = entry_rows.iloc[-1]
    exit_row = exit_rows.iloc[-1]

    buy_price = float(entry_row["Close"])
    sell_price = float(exit_row["Close"])

    pnl = sell_price - buy_price

    results.append({
        "Date": d,
        "BuyPrice": round(buy_price, 4),
        "SellPrice": round(sell_price, 4),
        "PnL": round(pnl, 4)
    })

if not results:
    print("No trades found.")
else:
    results_df = pd.DataFrame(results)

    print("\nDaily P/L:")
    print(results_df.to_string(index=False))

    print("\nOverall P/L:", round(results_df["PnL"].sum(), 4))

C:\Users\swati\AppData\Local\Temp\ipykernel_1084\3906228903.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stock_name, interval="1m", period="7d")
[*********************100%***********************]  1 of 1 completed


Daily P/L:
      Date  BuyPrice  SellPrice   PnL
2025-11-17    973.55     973.00 -0.55
2025-11-18    972.70     971.90 -0.80
2025-11-19    983.50     982.75 -0.75
2025-11-20    980.00     981.00  1.00

Overall P/L: -1.1
